In [1]:
import selenium
from bs4 import BeautifulSoup
import requests
import re
import random
import os
from selenium import webdriver
import time
from PIL import Image
import io
import requests
import pandas as pd
from lxml import etree

In [2]:
class Data_Harvester:
    

    def __init__(self):  
        self.url_array=[] 
        self.final_array=[]
        
    #function to scrape the URLs of institutions from the first 40 pages of the site
    def get_urls(self):
       
        
        arr2=[]
        #Xpath of the URL
        xpath_string='//*[@id="page-top"]/main/article/div[1]/div/div[2]/div/div[1]/div/div[9]/div/h4/a';
        
        #Starting the driver
        driver = webdriver.Chrome()
        driver.get('https://www.grid.ac/institutes') 
        
        #Looping over the first 40 pages
        for j in range(40):
            
            xpath_string='//*[@id="page-top"]/main/article/div[1]/div/div[2]/div/div[1]/div/div[9]/div/h4/a';
            #Modifying the xpath string and fetching the URLs for the first 8 institutions
            for x in range(2,10):
                xpath_string=xpath_string[0:70]+str(x)+xpath_string[71:]
                elem=driver.find_elements_by_xpath(xpath_string)[0]
                href=elem.get_attribute('href')
                self.url_array.append(href)
             
            #Modifying the xpath string and fetching the URLs for the rest of the institutions
            xpath_string='//*[@id="page-top"]/main/article/div[1]/div/div[2]/div/div[1]/div/div[10]/div/h4/a';
            for x in range(10,27):
                xpath_string=xpath_string[0:70]+str(x)+xpath_string[72:]
                elem=driver.find_elements_by_xpath(xpath_string)[0]
                href=elem.get_attribute('href')
                self.url_array.append(href)
            
            #Clicking the 'Next' button and going to the next page    
            submit_button = driver.find_elements_by_xpath('//*[@id="page-top"]/main/article/div[1]/div/div[2]/div/div[2]/nav/span[10]/a')[0]
            submit_button.click()
            time.sleep(4)

              
        self.parse_urls()
        return self.final_array
            
        
        #Function to scrape parameters from link passed to it
    def get_params(self,link):
            
        #Initializing dictionary
        Params={
            'Institute_name':"",
            'Grid_id':"",
            'Type':"",
            'Established_year':"",
            'Institute_link':"",
            'Wikipedia_link':"",
            'Alias_list':"",
            'City':"",
            'City_geoNames_Code':"",
            'City_geoNames_id':"",
            'Country':"",
            'Country_geoNames_Code':"",
            'Country_geoNames_id':"",
            'NUTS1_name':"",
            'NUTS1_code':"",
            'NUTS2_name':"",
            'NUTS2_code':"",
            'NUTS3_name':"",
            'NUTS3_code':""
            }


        time.sleep(3)
        
        #Getting HTML from the webpage
        html_text=requests.get(link).text
        soup = BeautifulSoup(html_text, "html.parser")
        dom = etree.HTML(str(soup))
        
        #Scraping parameters
        try:
            Params['Institute_name']=dom.xpath('//*[@id="page-top"]/header/div/h1')[0].text
        except:
            pass
        try:
            Params['Grid_id']=dom.xpath('//*[@id="institute-grid-id"]')[0].text
        except:
            pass
        try:
            Params['Type']=dom.xpath('//*[@id="page-top"]/main/article/div/div[1]/div[1]/div[1]/div/div/div[2]/div/ul/li')[0].text.replace("\n","").strip()
        except:
            pass
        try:
            Params['Established_year']=dom.xpath('//*[@id="page-top"]/main/article/div/div[1]/div[1]/div[1]/div/div/div[3]/div/span')[0].text.strip()
        except:
            pass
        try:
            Params['Wikipedia_link']=dom.xpath('//*[@id="page-top"]/main/article/div/div[1]/div[1]/div[2]/div/div/div[2]/div/a')[0].text.strip()
        except:
            pass
        try:
            Params['Institute_link']=dom.xpath('//*[@id="page-top"]/main/article/div/div[1]/div[1]/div[2]/div/div/div[1]/div/ul/li/a')[0].text.strip()
        except:
            pass
        try:
            alias=soup.find_all("li",itemprop="alternateName")
            alias_list=""
            for alt in alias:
                alias_list= alias_list+ alt.text.strip()+","
            Params['Alias_list']=alias_list
        except:
            pass
        try:
            Params['City']=soup.find(text="City").find_next('td').contents[0]
            Params['City_geoNames_Code']=soup.find(text="City").find_next('td').find_next('td').text.strip()
            Params['City_geoNames_id']=soup.find(text=City).find_next('td').find_next('td').text.strip()
        except:
            pass
        try:
            Params['NUTS1_name']=soup.find(text="NUTS 1").find_next('td').contents[0]
            Params['NUTS1_code']=soup.find(text="NUTS 1").find_next('td').find_next('td').text.strip()
        except:
            pass
        try:
            Params['NUTS2_name']=soup.find(text="NUTS 2").find_next('td').contents[0]
            Params['NUTS2_code']=soup.find(text="NUTS 2").find_next('td').find_next('td').text.strip()
        except:
            pass
        try:
            Params['NUTS3_name']=soup.find(text="NUTS 3").find_next('td').contents[0]
            Params['NUTS3_code']=soup.find(text="NUTS 3").find_next('td').find_next('td').text.strip()
        except:
            pass
        try:
            Params['Country']=soup.find(text="Country/Territory").find_next('td').contents[0]
            Params['Country_geoNames_Code']=soup.find(text="Country/Territory").find_next('td').find_next('td').text.strip()
            Params['Country_geoNames_id']=soup.find(text=Country).find_next('td').find_next('td').text.strip()
        except:
            pass
    
        #Appending dictionary to array
        self.final_array.append(Params)

    #Looping over each URL and fetching paramters   
    def parse_urls(self):
        for i in self.url_array:
            self.get_params(i)
        

In [3]:
p= Data_Harvester()
arr=p.get_urls()

In [6]:
#Converting to dataframe
df = pd.DataFrame(arr)

#To scv
df.to_csv('Institute_data.csv', header=True, index=False)

In [8]:
df.head()

,Institute_name,Grid_id,Type,Established_year,Institute_link,Wikipedia_link,Alias_list,City,City_geoNames_Code,City_geoNames_id,Country,Country_geoNames_Code,Country_geoNames_id,NUTS1_name,NUTS1_code,NUTS2_name,NUTS2_code,NUTS3_name,NUTS3_code
0,National Natural Science Foundation of China,grid.419696.5,Government,1986 CE,http://www.nsfc.gov.cn/publish/portal1/,https://en.wikipedia.org/wiki/National_Natural...,"NSFC,",Beijing,,,China,CN,,,,,,,
1,University of Toronto,grid.17063.33,Education,1827 CE,http://www.utoronto.ca/,https://en.wikipedia.org/wiki/University_of_To...,,Toronto,,,Canada,CA,,,,,,,
2,Samsung (South Korea),grid.419666.a,Company,1938 CE,http://www.samsung.com/sec/home/,https://en.wikipedia.org/wiki/Samsung,,Seoul,,,South Korea,KR,,,,,,,
3,Ministry of Science and Technology of the Peop...,grid.424020.0,Government,1998 CE,http://www.most.gov.cn/eng/,https://en.wikipedia.org/wiki/Ministry_of_Scie...,"State Science and Technology Commission,MOST,",Beijing,,,China,CN,,,,,,,
4,United States Department of the Navy,grid.420434.5,Government,1798 CE,http://www.navy.mil/,https://en.wikipedia.org/wiki/United_States_De...,"DON,",Arlington,,,United States,US,,,,,,,
